In [19]:
# import cv2
# import matplotlib.pyplot as plt
# import numpy as np
# import utils.image_proc as ip
# import utils.function as f
# from utils.decode import EAN13DECODER
# from utils.decode import DECODEDONE

# %load_ext autoreload
# %autoreload 2

In [20]:
# # img = cv2.imread("../img/barcode13.jpg")
# img = cv2.imread("./test2.bmp")
# # img = cv2.resize(img, (300, 400))
# img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# img = ip.normalize(img)
# img = ip.contrast(img, 0.8)
# img = ip.gaussian_blur(img)

# mean,std = cv2.meanStdDev(img)
# # raise up the value to keep more black pixels
# th = 0.5
# img = ip.binarize(img, th)
# img = 1 - img
# line_num = 20
# angle = 90
# # 0<=angle<=180
# x,y,lines = f.apply_scanline(img, line_num, angle, resolution=1000)

# plt.imshow(img, cmap="gray")
# plt.scatter(x, y, c="r", s=2)


# for x_, y_ in zip(x, y):
#     xx = np.linspace(0, img.shape[1], 100)
#     yy = y_ + (xx - x_) * np.tan((angle-90) / 180 * np.pi)
#     plt.plot(xx, yy, c="r")

# plt.show()
# np.save("lines.npy", lines)

In [21]:
# import seaborn as sns
# import pandas as pd

# barcodes = []
# with open('./barcodes.txt', 'r') as f:
#     for line in f:
#         barcodes.append(line.strip())

# all_interv = []
# for barcode in barcodes:
#     interv_count = np.zeros(6)
#     barcode_nplist = np.array([int(i) for i in barcode])
#     decoder = EAN13DECODER()
#     intervals, _ = decoder.get_intervals(barcode=barcode_nplist)

#     for i in intervals:
#         interv_count[i] += 1

#     for i in range(6):
#         all_interv.append({
#             "freq": i,
#             "count": interv_count[i],
#         })

# all_interv = pd.DataFrame(all_interv)
# sns.set_style("whitegrid")
# sns.barplot(x="freq", y="count", data=all_interv)

# plt.xticks(np.arange(6))
# plt.xlim(0.5, 4.5)


![](./簡報1.png)


In [2]:
from utils.slover import solver

%load_ext autoreload
%autoreload 2

In [31]:
line_num = 100
# 0<=angle<=180
angles = [ 90, 60, 120, 30, 150, 0, 180]
# angles = [90]
# tilts = [None, 3, -3]
tilts = [None]
torelances = [2, 3, 4, 5, 1]
# torelances = [5]


acc,duration,all_codes,all_imgs = solver('./barcodes', line_num=line_num, angles=angles, tilts=tilts, torelances=torelances, show_res=False, show_drop=False)
print('accuracy:', acc)
print('avg time (sec):', duration, end='\n\n')
for i in range(len(all_codes)):
    print(all_imgs[i],':', all_codes[i])

  0%|          | 0/6 [00:00<?, ?it/s]

100%|██████████| 6/6 [00:11<00:00,  1.84s/it]

accuracy: 0.8333333333333334
avg time (sec): 1.8434115250905354

test1.bmp : 9780137050512
barcode03.jpg : fail
test2.bmp : 4710777110166
barcode13.jpg : 8002206000015
barcode17.jpg : 3014260257927
test8.jpg : 4902030187590
